In [2]:
import numpy as np
import pandas as pd
import math
import string
from nltk.stem import SnowballStemmer
from nltk.corpus import stopwords
from sklearn import preprocessing
from sklearn.model_selection import cross_val_score
from sklearn.preprocessing import LabelEncoder
from sklearn.decomposition import PCA, FastICA
from sklearn.decomposition import TruncatedSVD 
from sklearn.random_projection import GaussianRandomProjection
from sklearn.random_projection import SparseRandomProjection
from sklearn.feature_extraction import DictVectorizer
# from xgboost.sklearn import XGBClassifier
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import accuracy_score

from scipy.io import wavfile
import matplotlib.pyplot as plt
import matplotlib.pyplot as plt
%matplotlib inline
from scipy.signal import decimate
from sklearn.model_selection import train_test_split

from keras.models import Sequential
from keras.layers import Conv1D, MaxPool1D, GlobalAvgPool1D, Dropout, BatchNormalization, Dense
from keras.optimizers import Adam
from keras.callbacks import ModelCheckpoint, LearningRateScheduler, EarlyStopping
from keras.utils import np_utils
from keras.regularizers import l2

Using TensorFlow backend.


In [3]:
from sklearn.preprocessing import LabelEncoder
from sklearn.impute import SimpleImputer
import time
start = time.time()

meta = pd.read_csv("./AutoKaggle - Metadata.csv", encoding='cp1252')
arrOfRows = [64,360,239,198]
nlp_rows = [239]
cnn_rows = [196]
row = 196
train = ''

INPUT_LIB = './kinguistics_heartbeat-sounds/data/'
SAMPLE_RATE = 44100
CLASSES = ['artifact', 'normal', 'murmur']
CODE_BOOK = {x:i for i,x in enumerate(CLASSES)}   
NB_CLASSES = len(CLASSES)
def preprocessing(row):
    find_row = meta.loc[row]
    train = ''
    test = None
    check_test = True
    train_X = ''
    train_Y = ''
    test_X = None
    if meta['name'].loc[row] == 'kobe-bryant-shot-selection':
        train = pd.read_csv("./kobe-bryant-shot-selection/data/data.csv", encoding='cp1252')
        check_test = False
    elif meta['name'].loc[row] == 'mercedes-benz-greener-manufacturing':
        train = pd.read_csv("./mercedes-benz-greener-manufacturing/data/train.csv")
        test = pd.read_csv("./mercedes-benz-greener-manufacturing/data/test.csv")
    elif meta['name'].loc[row] == 'uciml_sms-spam-collection-dataset':
        train = pd.read_csv("./uciml_sms-spam-collection-dataset/data/spam.csv", encoding='cp1252', error_bad_lines=False)
        check_test = False
    elif meta['name'].loc[row] == 'kinguistics_heartbeat-sounds':
        train = []
        train.append(pd.read_csv("./kinguistics_heartbeat-sounds/data/set_a_timing.csv"))
        train.append(pd.read_csv("./kinguistics_heartbeat-sounds/data/set_a.csv"))
        train.append(pd.read_csv("./kinguistics_heartbeat-sounds/data/set_b.csv"))
        check_test = False
        
    
    if check_test:
        test = test.dropna()
    if meta['name'].loc[row] == 'uciml_sms-spam-collection-dataset':
        row = pd.read_csv("./uciml_sms-spam-collection-dataset/submission/row.csv", encoding='cp1252')
        sms = train
        row_prepro = row['preprocessing function call'][0]
        prepro_ls = eval(row_prepro)
        sms = eval(prepro_ls[0])
        train = eval(prepro_ls[1])
        return train
    
    
    
    
    
    
    elif meta['name'].loc[row] == 'kinguistics_heartbeat-sounds':
        row = pd.read_csv("./kinguistics_heartbeat-sounds/submission/row.csv", encoding='cp1252')
        prepro = eval(row['preprocessing function call'][0])
        eval(prepro[0])
        apply = eval(row['apply'][0])
        apply_column = eval(row['apply_column'][0])
        apply = eval(row['apply'][0])
        apply_target = eval(row['apply_target'][0])
        for i in range(len(apply)):
            eval_str = "train[1][" + apply_target[i] + "] = train[1][" + apply_column[i] + "].apply(" + apply[i] + ")"
            exec(eval_str)
            
        x_data = np.stack(train[1]['time_series'].values, axis=0)
        new_labels =[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
             0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 2, 1, 1, 1,
             1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 2, 1, 1, 1, 2, 2, 2, 1, 1, 2, 1, 
             2, 2, 1, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 
             2, 2, 2, 2, 2, 1, 1, 1, 1, 1, 1, 1, 1, 2, 1, 1, 1, 1, 1, 2, 1, 1, 
             1, 1, 1, 1, 1, 2, 2, 1, 1, 1, 1, 1, 2, 1, 0, 2, 2, 1, 1, 1, 1, 1, 
             0, 1, 0, 1, 1, 1, 2, 1, 0, 1, 1, 1, 1, 1, 2, 0, 0, 0, 0, 0, 0, 0, 
             1, 0, 0, 0, 0, 0, 0, 1, 0, 2, 1, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2]
        new_labels = np.array(new_labels, dtype='int')
        y_data = np_utils.to_categorical(new_labels)
        x_train, x_test, y_train, y_test, train_filenames, test_filenames = \
        train_test_split(x_data, y_data, train[1]['fname'].values, test_size=0.25)
        x_train = decimate(x_train, 8, axis=1, zero_phase=True)
        x_train = decimate(x_train, 8, axis=1, zero_phase=True)
        x_train = decimate(x_train, 4, axis=1, zero_phase=True)
        x_test = decimate(x_test, 8, axis=1, zero_phase=True)
        x_test = decimate(x_test, 8, axis=1, zero_phase=True)
        x_test = decimate(x_test, 4, axis=1, zero_phase=True)
        x_train = x_train / np.std(x_train, axis=1).reshape(-1,1)
        x_test = x_test / np.std(x_test, axis=1).reshape(-1,1)
        x_train = x_train[:,:,np.newaxis]
        x_test = x_test[:,:,np.newaxis]
        return x_train, x_test, y_train, y_test
    else:
        train = train.dropna()
        for c in train.columns:
            if train[c].dtype == 'object':    # deal with text
                lbl = LabelEncoder() 
                if check_test:
                    lbl.fit(list(train[c].values) + list(test[c].values)) 
                    train[c] = lbl.transform(list(train[c].values))
                    test[c] = lbl.transform(list(test[c].values))
                else:
                    lbl.fit(list(train[c].values))
                    train[c] = lbl.transform(list(train[c].values))

        targetName = find_row['targetName']
        train_Y = train[targetName]
        train_X = train.drop(columns=targetName)
        if check_test:
            test_X = test
            return train_X, train_Y, test_X
        else:
            return train_X, train_Y, None

        
def clean_filename(fname, string):   
    file_name = fname.split('/')[1]
    if file_name[:2] == '__':  
        file_name = string + file_name
    return file_name

def load_wav_file(name, path):
    _, b = wavfile.read(path + name)
    assert _ == SAMPLE_RATE
    return b

def repeat_to_length(arr, length):
    """Repeats the numpy 1D array to given length, and makes datatype float"""
    result = np.empty((length, ), dtype = 'float32')
    l = len(arr)
    pos = 0
    while pos + l <= length:
        result[pos:pos+l] = arr
        pos += l
    if pos < length:
        result[pos:length] = arr[:length-pos]
    return result


print(meta['name'].loc[row])


nan


In [4]:
def text_process(text):
    
    text = text.translate(str.maketrans('', '', string.punctuation))
    text = [word for word in text.split() if word.lower() not in stopwords.words('english')]
    
    return " ".join(text)

def feature_extraction(row, X_train, X_test):
    if meta['name'].loc[row] == 'uciml_sms-spam-collection-dataset':
        rowcsv = pd.read_csv("./uciml_sms-spam-collection-dataset/submission/row.csv")
        row_extract = rowcsv['featureExtractor function call'].loc[0]
        sms = X_train
        extract = eval(row_extract)
        sms['message'] = eval(extract[0])
        sms['message'] = eval(extract[1])
        text_feat = sms['message'].apply(str).copy()
        text_feat = eval(extract[2])
        vectorizer = eval(extract[3])
        features = eval(extract[4])
        features_train, features_test, labels_train, labels_test = train_test_split(features, sms['label'], test_size=0.3)
        return features_train, features_test, labels_train, labels_test
    else:
        if type(meta["featureExtractor function call"].loc[row]) is not str:
            print('not func')
            return X_train,X_test
        extraction_function_calls = str(meta["function call feature extraction"].loc[row])
        extraction_function_calls = extraction_function_calls.split(",")
        extraction_funtion_param = eval(meta["function parameters feature extraction"].loc[row])
        function_nums = len(extraction_function_calls)
        for i in range(function_nums):
            str1 = extraction_function_calls[i]
            str2 = extraction_funtion_param[i]
            l_str = str1.split("(")
            l_str.insert(1,"("+str2)
            str_call = ''
            str_call = str_call.join(l_str)
            str_call = 'extractor' + '=' + str_call
            exec(str_call, globals(), globals())
            extracted_train = extractor.fit_transform(X_train)
            n_comp = extracted_train.shape[1]
            for j in range(0, n_comp):
                X_train['extractor'+ str(i)+"_"+str(j)] = extracted_train[:, j]
            if X_test is not None:
                extracted_test = extractor.fit_transform(X_test)
                for j in range(0, n_comp):
                    X_test['extractor'+ str(i)+"_"+str(j)] = extracted_test[:, j]
                return X_train,X_test
            else:
                return X_train,None

In [5]:
train = preprocessing(row)
feature_extraction(row, train, None)

AttributeError: 'NoneType' object has no attribute 'dropna'

In [ ]:
def feature_selection():
    pass

In [ ]:
def batch_generator(x_train, y_train, batch_size):
    """
    Rotates the time series randomly in time
    """
    x_batch = np.empty((batch_size, x_train.shape[1], x_train.shape[2]), dtype='float32')
    y_batch = np.empty((batch_size, y_train.shape[1]), dtype='float32')
    full_idx = range(x_train.shape[0])
    
    while True:
        batch_idx = np.random.choice(full_idx, batch_size)
        x_batch = x_train[batch_idx]
        y_batch = y_train[batch_idx]
    
        for i in range(batch_size):
            sz = np.random.randint(x_batch.shape[1])
            x_batch[i] = np.roll(x_batch[i], sz, axis = 0)
     
        yield x_batch, y_batch
def estimation(row,X_train,X_test,Y_train, Y_test):
    if row in nlp_rows:
        rowcsv = pd.read_csv("./uciml_sms-spam-collection-dataset/submission/row.csv")
        row_extract = eval(rowcsv['estimator1 function call'].loc[0])
        mnb = eval(row_extract[0])
        eval(row_extract[1])
        pred = eval(row_extract[2])
        if rowcsv['performanceMetric'].loc[0] == 'accuracy':
            return accuracy_score(Y_test, pred)
    elif row in cnn_rows:
        rowcsv = pd.read_csv("./kinguistics_heartbeat-sounds/submission/row.csv")
        cnn_callback = eval(rowcsv['cnn_callback'].loc[0])
        callback = [0, 0]
        for i in range(len(cnn_callback)):
            exec(cnn_callback[i])
        estimator1_function_call = eval(rowcsv['estimator1 function call'].loc[0])
        model = eval(estimator1_function_call[0])
        cnn_train = eval(rowcsv['cnn_train'].loc[0])
        for i in range(len(cnn_train)):
            exec("model." + cnn_train[i])
        scores = model.evaluate(X_test, Y_test, verbose=1)
        print('Test loss:', scores[0])
        print('Test accuracy:', scores[1])    
    else:
        estimation_function_calls = eval(meta["estimator1 function call"].loc[row])
        print(len(estimation_function_calls))
        if len(estimation_function_calls) == 1:
            str_call = estimation_function_calls[0]
            print(str_call)
            str_call = 'estimator' + '=' + str_call
            print(str_call)
            exec(str_call,globals(),globals())
            
            if meta["taskType"].loc[row] == 'classification':
                estimator.fit(X_train,Y_train)
                Y_pred = estimator.predict(X_test)
                print('here')
                print(recall_score(Y_test,Y_pred,average='weighted'))
                print('here')
            elif meta["taskType"].loc[row] == 'regression':
                estimator.fit(X_train,Y_train)
                print(r2_score(Y_test,Y_pred,average='weighted'))
        else:
            estimators = []
            n_estimators = len(estimation_function_calls)
            for i in range(n_estimators):
                str1 = extraction_function_calls
                str2 = extraction_funtion_param
                l_str = str1.split("(")
                l_str.insert(1,"("+str2)
                str_call = ''
                str_call = str_call.join(l_str)
                str_call = 'estimator' + '=' + str_call
                print(l_str)
                print(str_call)
                exec(str_call)
                estimators.append(estimator)
                postprocessing(estimators,stack = True)

In [ ]:
train_set = preprocessing(row)
X_train, X_test, Y_train, Y_test = feature_extraction(row, train_set, None)
Y_pred = estimation(row, ['shot_made_flag'], None, None, None)
print(Y_pred)

In [ ]:
if meta['name'].loc[row] == 'uciml_sms-spam-collection-dataset':
    train_set = preprocessing(row)
    X_train, X_test, Y_train, Y_test = feature_extraction(row, train_set, None)
    Y_pred = estimation(row, X_train, X_test, Y_train, Y_test)
    print(Y_pred)
    
if meta['name'].loc[row] == 'kinguistics_heartbeat-sounds':
    X_train, X_test, Y_train, Y_test = preprocessing(row)
    estimation(row,X_train,X_test,Y_train, Y_test)

In [ ]:
def postprocessing(estimators,stack):
    pass

In [6]:
end = time.time()
print("Use", end - start, "seconds to run this.")

Use 1.539588212966919 seconds to run this.
